In [1]:
!pip install langchain langchain_community --quiet

In [2]:
!pip install --upgrade openai

In [3]:
import openai
from langchain.llms import OpenAI
import os

In [4]:
from langchain_community.llms import OpenAI as LangChainOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import List
import json

In [5]:
from google.colab import userdata
open_ai_api_key = userdata.get('OPENAI_API_KEY')
hugging_face_token = userdata.get('HF_TOKEN')

In [6]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_api_key

In [7]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
# Initialize OpenAI client
client = OpenAI(api_key=open_ai_api_key)

# Initialize LangChain LLM
llm = LangChainOpenAI(temperature=0)

<ipython-input-8-9f17bca48abe>:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=open_ai_api_key)


## **1. In-Context Learning (ICL)**
Success Case:

In [12]:
# Example of ICL
prompt = (
    "Translate the following sentences into French:\n"
    "1. I love programming. -> J'adore la programmation.\n"
    "2. How are you? -> Comment ça va?\n"
    "3. This is a test. ->"
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Success Case:", response.choices[0].message.content.strip())



Success Case: C'est un test.


Failure Case:

In [19]:
# Example of ICL Failure (ambiguous examples or insufficient context)
prompt = (
    "Translate to French:\n"
    "1. Hello -> Bonjour\n"
    "2. Bye -> Au revoir\n"
    "3. How do you do? ->"
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Failure Case:", response.choices[0].message.content.strip())


Failure Case: Comment ça va ?


## **2. Chain of Thought (CoT)**
Success Case:

In [20]:
# CoT Example - Encourage step-by-step reasoning
prompt = (
    "Q: If you have 2 apples and you buy 3 more, how many apples do you have in total?\n"
    "A: Let's think step by step."
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Success Case:", response.choices[0].message.content.strip())

Success Case: Step 1: Start with 2 apples.
Step 2: Buy 3 more apples.
Step 3: Add the 3 apples to the 2 apples you already have.
Step 4: 2 apples + 3 apples =


Failure Case:

In [21]:
# CoT Failure - Skips reasoning steps due to insufficient guidance
prompt = "If you have 2 apples and buy 3 more, how many apples do you have?"
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Failure Case:", response.choices[0].message.content.strip())


Failure Case: You would have 5 apples.


## **3. Iterative Chain of Thought (iCoT)**
Success Case:

In [35]:
# iCoT Example
steps = [
    "Step 1: Solve the math problem 12 + 15.",
    "Step 2: Double the result of Step 1.",
    "Step 3: Add 10 to the result of Step 2.",
    "Step 4: What is the final answer?"
]
results = []
for step in steps:
    MODEL = "gpt-3.5-turbo"
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": step}],
        max_tokens=50,
        temperature=0
    )
    results.append(response.choices[0].message.content.strip())

    print(f"{step}: {response.choices[0].message.content.strip()}")


Step 1: Solve the math problem 12 + 15.: 12 + 15 = 27.
Step 2: Double the result of Step 1.: Let's say the result of Step 1 was x. 

Step 2: Double the result of Step 1

2x
Step 3: Add 10 to the result of Step 2.: Let's say the result of Step 2 is 20. 

Adding 10 to 20 would give us:

20 + 10 = 30

So, the result of adding 10 to the result of Step 2 is 30
Step 4: What is the final answer?: The final answer will depend on the specific question or problem that was being solved. Without that information, it is not possible to provide a final answer.


Failure Case:

In [34]:
# iCoT Failure (skips interdependencies between steps)
steps = [
    "Solve 12 + 15.",
    "Double the previous result.",
    "Add 10 to the last result."
]
for step in steps:
    MODEL = "gpt-3.5-turbo"
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": step}],
        max_tokens=50,
        temperature=0
    )
    print(f"Failure Case: {step}: {response.choices[0].message.content.strip()}")


Failure Case: Solve 12 + 15.: 12 + 15 = 27
Failure Case: Double the previous result.: If the previous result was x, then the new result would be 2x.
Failure Case: Add 10 to the last result.: The last result was 20. Adding 10 to that gives us 30.


## **4. Tree of Thought (ToT)**
Success Case:

In [36]:
llm = OpenAI(temperature=0.7)
template = PromptTemplate(
    input_variables=['problem'],
    template="""
    Let's solve this step by step and consider multiple possibilities:
    {problem}
    """
)
chain = LLMChain(llm=llm, prompt=template)
response = chain.run(problem="What are the factors of 28?")
print("Success Case:", response)


<ipython-input-36-5ce43d1ff531>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=template)
<ipython-input-36-5ce43d1ff531>:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(problem="What are the factors of 28?")


Success Case: 1. 1 x 28
    2. 2 x 14
    3. 4 x 7

    So, there are three pairs of factors for 28. Now, let's check if any of these pairs are prime numbers.
    1. 1 x 28 - 1 is not a prime number as it is a composite number.
    2. 2 x 14 - 2 is a prime number.
    3. 4 x 7 - 4 is not a prime number as it is a composite number.

    Therefore, out of the three pairs of factors of 28, only one pair (2 x 14) consists of a prime number (2).

    Hence, the prime factors of 28 are 2 and 14.


Failure Case:

In [38]:
# ToT Failure - Insufficient reasoning
prompt = "What are the factors of 28?"
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Failure Case:", response.choices[0].message.content.strip())


Failure Case: The factors of 28 are: 1, 2, 4, 7, 14, and 28.


## **5. Graph of Thought (GoT)**
Success Case:

In [39]:
# GoT Example - Interrelated solutions
prompt = (
    "Solve these related problems:\n"
    "1. What is 12 + 8?\n"
    "2. What is 20 divided by the answer to 1?\n"
    "3. Multiply the answer to 2 by 4.\n"
    "Show all steps."
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Success Case:", response.choices[0].message.content.strip())


Success Case: 1. 12 + 8 = 20
2. 20 divided by 20 = 1
3. 1 x 4 = 4

Therefore, the final answer is 4.


Failure Case:

In [40]:
# GoT Failure - Lack of dependency tracking
prompt = (
    "Solve these problems:\n"
    "1. Add 12 and 8.\n"
    "2. Divide 20 by 4.\n"
    "3. Multiply 5 by 3."
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Failure Case:", response.choices[0].message.content.strip())

Failure Case: 1. 12 + 8 = 20
2. 20 ÷ 4 = 5
3. 5 x 3 = 15


## **6. Agent of Thought (AoT)**
Success Case:

In [41]:
# Simulating an agent collaborating on tasks
prompts = [
    "Agent A: Find the capital of France.",
    "Agent B: Cross-reference the population of the city.",
    "Agent A: Add historical context about its landmarks."
]
for prompt in prompts:
    MODEL = "gpt-3.5-turbo"
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0
    )
    print(f"{prompt}: {response.choices[0].message.content.strip()}")


Agent A: Find the capital of France.: The capital of France is Paris.
Agent B: Cross-reference the population of the city.: Agent A: The population of the city is approximately 500,000 according to the latest census data.
Agent A: Add historical context about its landmarks.: The landmarks in this area hold significant historical importance, dating back to the early 19th century. The iconic clock tower was built in 1820 as a symbol of the town's prosperity during the Industrial Revolution. The grand cathedral, constructed in


## **7. REACT (Reasoning and Acting)**
Success Case:

In [42]:
# REACT Example - Reasoning and acting in tandem
prompt = (
    "Question: Who is the President of the United States?\n"
    "Thought: I need to retrieve the latest data.\n"
    "Action: Use external API to find the current President.\n"
    "Observation: The President is Joe Biden.\n"
    "Final Answer: Joe Biden."
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Success Case:", response.choices[0].message.content.strip())


Success Case: President Joe Biden is the current President of the United States.


## **8. RASCEF**
Success Case:

In [43]:
# RASCEF Example - Building structured reasoning
prompt = (
    "Problem: How do you determine if a number is prime?\n"
    "Reason: A number is prime if it has no divisors other than 1 and itself.\n"
    "Evidence: Check divisibility up to the square root of the number.\n"
    "Framework: Write a Python function to validate the reasoning.\n"
)
MODEL = "gpt-3.5-turbo"
response = openai.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=50,
    temperature=0
)
print("Success Case:", response.choices[0].message.content.strip())



Success Case: ```python
import math

def is_prime(num):
    if num <= 1:
        return False
    if num == 2:
        return True
    if num % 2 == 0:
        return False
    
    for i


## **9. Forest of Thoughts**
Success Case:

In [44]:
# Forest of Thoughts using LangChain
llm = OpenAI(temperature=0.7)
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="Explore various solutions for: {topic}"
)
chain = LLMChain(llm=llm, prompt=prompt_template)
response = chain.run(topic="Climate Change Mitigation Strategies")
print("Success Case:", response)


Success Case: 

1. Renewable Energy Sources
One key solution for mitigating climate change is to transition to renewable energy sources, such as solar, wind, hydropower, and geothermal energy. These sources emit little to no greenhouse gases and can replace fossil fuels in powering homes, businesses, and transportation.

2. Energy Efficiency
Improving energy efficiency in buildings, industries, and transportation can also significantly reduce greenhouse gas emissions. This can be achieved through measures such as better insulation, efficient lighting and appliances, and more fuel-efficient vehicles.

3. Carbon Pricing
Implementing a price on carbon emissions, either through a carbon tax or a cap-and-trade system, can incentivize industries and individuals to reduce their carbon footprint. This can also generate revenue that can be reinvested in clean energy and climate change adaptation measures.

4. Sustainable Land Use Practices
Deforestation and land use change are major contributor